# 1. Create SQL database for traffic crash datasets

In [1]:
#import libraries 
import sqlite3
import pandas as pd


In [2]:
#list the csv files
csv_files = [
    ('Data/Traffic_Crashes_Resulting_in_Fatality_20240819.csv', 'traffic_crash_fatality'),
    ('Data/Traffic_Crashes_Resulting_in_Injury__Parties_Involved_20240819.csv', 'traffic_crash_parties'),
    ('Data/Traffic_Crashes_Resulting_in_Injury__Victims_Involved_20240819.csv', 'traffic_crash_victims'),
    ('Data/Traffic_Crashes_Resulting_in_Injury_20240819.csv', 'traffic_crash_all')
]

#create/connect to sqlite database
conn = sqlite3.connect("traffic_crash.db")

#import the data into the correct table
for csv_file, table_name in csv_files:
    #read the file
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)

#close the connect
conn.close()

c:\Users\yong2\miniconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (3,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\yong2\miniconda3\envs\py37\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


# 2. Data Wrangling

In [2]:
%load_ext sql
%sql sqlite:///traffic_crash.db

#### **ISSUE #1: There should only be one case_id_pkey for each incident in traffic_crash_all, but case_id_pkey 200376243 has 2 entries. Remove one row corresponding to case_id_pkey 200376243, since the information is the same.** 

In [18]:
%%sql

SELECT
    case_id_pkey, 
    count(*) AS n_incidents 
FROM 
    traffic_crash_all
GROUP BY 
    case_id_pkey
HAVING 
    n_incidents > 1

 * sqlite:///traffic_crash.db
Done.


case_id_pkey,n_incidents
200376243,2


In [19]:
%%sql

SELECT * 
FROM 
    traffic_crash_all
WHERE 
    case_id_pkey = 200376243

 * sqlite:///traffic_crash.db
Done.


unique_id,cnn_intrsctn_fkey,cnn_sgmt_fkey,case_id_pkey,tb_latitude,tb_longitude,geocode_source,geocode_location,collision_datetime,collision_date,collision_time,accident_year,month,day_of_week,time_cat,juris,officer_id,reporting_district,beat_number,primary_rd,secondary_rd,distance,direction,weather_1,weather_2,collision_severity,type_of_collision,mviw,ped_action,road_surface,road_cond_1,road_cond_2,lighting,control_device,intersection,vz_pcf_code,vz_pcf_group,vz_pcf_description,vz_pcf_link,number_killed,number_injured,street_view,dph_col_grp,dph_col_grp_description,party_at_fault,party1_type,party1_dir_of_travel,party1_move_pre_acc,party2_type,party2_dir_of_travel,party2_move_pre_acc,point,data_as_of,data_updated_at,data_loaded_at,analysis_neighborhood,supervisor_district,police_district,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Neighborhoods,SF Find Neighborhoods
61456,25723000.0,4013000.0,200376243,37.7629445652,-122.42851403699996,SFPD-INTERIM DB,CITY STREET,06/23/2020 10:31:00 AM,2020 June 23,10:31:00,2020,June,Tuesday,10:01 am to 2:00 pm,3801,1023,Mission,1-Car,CHURCH ST,17TH ST,20.0,North,Clear,Not Stated,Injury (Other Visible),Overturned,Not Stated,No Pedestrian Involved,Dry,No Unusual Condition,Not Stated,Daylight,Functioning,Intersection <= 20ft,22350,22350,Unsafe speed for prevailing conditions,http://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=VEH&sectionNum=22350,0.0,1,"https://maps.google.com/maps?q=&layer=c&cbll=37.7629445652,-122.42851403699997",FF,Bicycle Only,None,Bicyclist,North,Proceeding Straight,None,None,None,POINT (-122.428514037 37.762944565),05/03/2023 12:00:00 AM,04/29/2024 12:00:00 AM,07/12/2024 03:08:07 PM,Castro/Upper Market,8.0,MISSION,3.0,5.0,5.0,37.0,37.0
53071,25723000.0,4013000.0,200376243,37.7629445652,-122.42851403699996,SFPD-INTERIM DB,CITY STREET,06/23/2020 10:31:00 AM,2020 June 23,10:31:00,2020,June,Tuesday,10:01 am to 2:00 pm,3801,1023,Mission,1-Car,CHURCH ST,17TH ST,20.0,North,Clear,Not Stated,Injury (Other Visible),Overturned,Not Stated,No Pedestrian Involved,Dry,No Unusual Condition,Not Stated,Daylight,Functioning,Intersection <= 20ft,22350,22350,Unsafe speed for prevailing conditions,http://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=VEH&sectionNum=22350,0.0,1,"https://maps.google.com/maps?q=&layer=c&cbll=37.7629445652,-122.428514037",FF,Bicycle Only,None,Bicyclist,North,Proceeding Straight,None,None,None,POINT (-122.428514037 37.762944565),06/23/2020 12:00:00 AM,04/26/2023 12:00:00 AM,07/12/2024 03:08:07 PM,Castro/Upper Market,8.0,MISSION,3.0,5.0,5.0,37.0,37.0


#### **ISSUE #2: accident_year in traffic_crash_all is inaccurate (unless someone predicted the future). We are in 2024, and there are incidents in 2025 and 2026**

In [20]:
%%sql

SELECT 
    accident_year, 
    COUNT(*) as n_incidents 
FROM 
    traffic_crash_all
GROUP BY 
    accident_year
HAVING 
    accident_year > 2024


 * sqlite:///traffic_crash.db
Done.


accident_year,n_incidents
2025,1
2026,1


#### **SOLUTION: Create a view of traffic_crash_all where we remove the extra entry for case_id_pkey 200376243, remove accident_year, and create a new column for the year.** 

In [21]:
%%sql

DROP VIEW IF EXISTS view_traffic_crash_all;

CREATE VIEW view_traffic_crash_all AS
SELECT
    unique_id,
    case_id_pkey,
    CAST(SUBSTR(collision_date, 1, 4) AS INTEGER) AS incident_year,
    collision_date,
    collision_time,
    collision_datetime,
    type_of_collision,
    number_killed,
    number_injured,
    dph_col_grp_description,
    primary_rd,
    secondary_rd,
    road_surface,
    lighting,
    analysis_neighborhood,
    tb_latitude,
    tb_longitude
FROM 
    traffic_crash_all
WHERE 
    unique_id != 61456 and incident_year!=2024;


 * sqlite:///traffic_crash.db
Done.
Done.


[]

# 3. Data Exploration

#### **QUESTION 1: For each year, output the total number of car incidents, number killed and number injuries.**


In [22]:
%%sql

SELECT
    incident_year,
    COUNT(*) AS incidents_per_year, 
    SUM(number_killed) AS number_killed_per_year,   
    SUM(number_injured) AS number_injured_per_year  
FROM 
    view_traffic_crash_all  
GROUP BY 
    incident_year;

 * sqlite:///traffic_crash.db
Done.


incident_year,incidents_per_year,number_killed_per_year,number_injured_per_year
2005,3257,27.0,4437
2006,2896,30.0,3806
2007,3065,42.0,3957
2008,3039,27.0,3864
2009,2900,31.0,3702
2010,3095,22.0,3942
2011,3097,28.0,3842
2012,3270,29.0,4018
2013,3071,35.0,3854
2014,2967,31.0,3675


#### **QUESTION #2: What is the annual average number of incidents, number killed and number injured? Exclude incidents in 2024 to a more accurate calcuation.**

In [23]:
%%sql

SELECT
    ROUND(AVG(incidents_per_year)) AS avg_incidents,
    ROUND(AVG(number_killed_per_year)) AS avg_number_killed,
    ROUND(AVG(number_injured_per_year)) AS avg_number_injured
FROM(
    SELECT
        incident_year,
        COUNT(*) AS incidents_per_year, 
        SUM(number_killed) AS number_killed_per_year,   
        SUM(number_injured) AS number_injured_per_year  
    FROM 
        view_traffic_crash_all
    GROUP BY 
        incident_year);

 * sqlite:///traffic_crash.db
Done.


avg_incidents,avg_number_killed,avg_number_injured
3068.0,29.0,3855.0


#### **QUESTION #3: Calculate the fraction of incidents for each unique crash grouping.**

In [24]:
%%sql

WITH calculate_n_incidents AS (
    SELECT COUNT(*) AS total_n_incidents
    FROM view_traffic_crash_all
),
crash_grouping AS (
    SELECT dph_col_grp_description AS crash_description, COUNT(*) AS n_incidents_per_group
    FROM view_traffic_crash_all
    GROUP BY dph_col_grp_description
)
SELECT 
    g.crash_description, 
    ROUND((g.n_incidents_per_group * 1.0)/c.total_n_incidents, 3) AS incidents_fraction
FROM   
    crash_grouping AS g, calculate_n_incidents AS c
ORDER BY 
    incidents_fraction DESC;


 * sqlite:///traffic_crash.db
Done.


crash_description,incidents_fraction
Vehicle(s) Only Involved,0.588
Vehicle-Pedestrian,0.243
Vehicle-Bicycle,0.135
Bicycle Only,0.018
Bicycle-Pedestrian,0.008
Bicycle-Parked Car,0.007
Pedestrian Only or Pedestrian-Parked Car,0.001
None,0.0
Bicycle-Unknown/Not Stated,0.0
Unknown/Not Stated,0.0


#### **QUESTION 4: What are the top ten streets (primary_rd) with the highest number of incidents?**

In [25]:
%%sql  

SELECT 
    primary_rd, 
    COUNT (*) AS n_incidents    
FROM 
    view_traffic_crash_all  
GROUP BY 
    primary_rd 
ORDER BY 
    n_incidents DESC   
LIMIT 10;

 * sqlite:///traffic_crash.db
Done.


primary_rd,n_incidents
MISSION ST,2495
MARKET ST,2091
VAN NESS AVE,1436
GEARY BLVD,1134
03RD ST,1072
19TH AVE,868
FULTON ST,845
ALEMANY BLVD,843
CALIFORNIA ST,823
FOLSOM ST,791


#### **QUESTION #5: Given that streets can span long distances and the primary_rd may not precisely locate incidents, we use the secondary_rd for better accuracy. What are the top ten streets (primary_rd and secondary_rd) with the highest number of incidents? The secondary_rd serves as a reference road from which distance and direct are measured.**

In [26]:
%%sql

SELECT 
    primary_rd, 
    secondary_rd,   
    COUNT (*) AS n_incidents    
FROM 
    view_traffic_crash_all  
GROUP BY    
    primary_rd, 
    secondary_rd    
ORDER BY 
    n_incidents DESC   
LIMIT 10;


 * sqlite:///traffic_crash.db
Done.


primary_rd,secondary_rd,n_incidents
MARKET ST,05TH ST,128
MARKET ST,OCTAVIA ST,113
MARKET ST,07TH ST,100
MARKET ST,GOUGH ST,97
JUNIPERO SERRA BLVD,19TH AVE,91
THE EMBARCADERO,WASHINGTON ST,91
VAN NESS AVE,HAYES ST,91
16TH ST,POTRERO AVE,80
UNABLE TO GEOCODE,None,78
MARKET ST,08TH ST,76


#### **QUESTION #6: Which road surface condition (e.g. wet, dry) has the highest number of incidents?**

In [27]:
%%sql

WITH road_surface_cte AS (
    SELECT 
        road_surface,
        COUNT(*) AS n_incidents
    FROM 
        view_traffic_crash_all  
    GROUP BY 
        road_surface
)
SELECT 
    road_surface,
    MAX(n_incidents) AS max_n_incidents 
FROM 
    road_surface_cte
ORDER BY 
    n_incidents DESC;



 * sqlite:///traffic_crash.db
Done.


road_surface,max_n_incidents
Dry,51260


#### **QUESTION #7: Determine the number of incidents for each hour of the day.**

In [28]:
%%sql

SELECT 
    strftime('%H', collision_time ) AS hour,
    COUNT(*) AS n_incidents
FROM 
    view_traffic_crash_all
GROUP BY 
    hour
ORDER BY 
    hour;


 * sqlite:///traffic_crash.db
Done.


hour,n_incidents
None,58
00,1234
01,1159
02,1195
03,563
04,417
05,574
06,1159
07,2194
08,3277


#### **QUESTION #8: Which lighting condition has the most incidents?**

In [29]:
%%sql

SELECT
    lighting,
    COUNT(*) as n_incidents
FROM 
    view_traffic_crash_all
GROUP BY
    lighting
ORDER BY
    n_incidents DESC;


 * sqlite:///traffic_crash.db
Done.


lighting,n_incidents
Daylight,37681
Dark - Street Lights,17470
Dusk - Dawn,2029
Not Stated,582
Dark - No Street Lights,418
Dark - Street Lights Not Functioning,104


#### **QUESTION #9: What is the average number of people who are injured?**

In [30]:
%%sql

SELECT
    ROUND(AVG(number_injured), 0) AS avg_n_injured
FROM 
    view_traffic_crash_all;


 * sqlite:///traffic_crash.db
Done.


avg_n_injured
1.0


#### **QUESTION #10: Show the incidents where number_injured >1 and number_killed is >0.**

In [31]:
%%sql

SELECT 
    case_id_pkey,
    number_injured,
    number_killed
FROM 
    view_traffic_crash_all
WHERE 
    number_injured > 1 AND number_killed > 0;

 * sqlite:///traffic_crash.db
Done.


case_id_pkey,number_injured,number_killed
131098641,2,1.0
3400212,2,1.0
190528318,4,1.0
5542002,7,1.0
3523326,2,2.0
220731586,3,1.0
3205271,2,1.0
2841564,2,1.0
160410228,3,1.0
3205255,2,1.0


#### **QUESTION #11: For the incidents that included deaths, output the incidents where the number of deaths >= 2.**

In [32]:
%%sql

SELECT 
    unique_id, 
    number_killed
FROM
    view_traffic_crash_all
WHERE 
    number_killed > 0
GROUP BY 
    number_killed
HAVING 
    number_killed >= 2;

 * sqlite:///traffic_crash.db
Done.


unique_id,number_killed
53747,2.0
36735,3.0


#### **QUESTION #12: Are there more male or female victims? Output the sex with the most number of injured people and the number of incidents.**

In [66]:
%%sql 

SELECT 
    party_sex, 
    COUNT(*) AS n_injured
FROM 
    traffic_crash_victims
WHERE
    CAST(SUBSTR(collision_date, 7, 4) AS INTEGER) < 2024
GROUP BY 
    party_sex
ORDER BY
    n_injured DESC;

 * sqlite:///traffic_crash.db
Done.


party_sex,n_injured
Male,46583
Female,26530
Not Stated,1053
Other,36


In [68]:
%%sql

SELECT
    party_sex, 
    MAX(n_injured) AS max_injured 
FROM
    (SELECT party_sex, COUNT(*) AS n_injured
    FROM traffic_crash_victims
    WHERE CAST(SUBSTR(collision_date, 7, 4) AS INTEGER) < 2024
    GROUP BY party_sex);

 * sqlite:///traffic_crash.db
Done.


party_sex,max_injured
Male,46583


#### **QUESTION #13: Are males or females more likely to be at fault?**

In [85]:
%%sql

SELECT
    party_sex, 
    COUNT(*) AS n_at_fault
FROM 
    (SELECT DISTINCT case_id_pkey, party_sex 
     FROM traffic_crash_victims
     WHERE at_fault = 'Yes' 
     AND CAST(SUBSTR(collision_date, 7, 4) AS INTEGER) < 2024
    ) AS distinct_parties
GROUP BY 
    party_sex
ORDER BY 
    n_at_fault DESC;


 * sqlite:///traffic_crash.db
Done.


party_sex,n_at_fault
Male,15411
Female,7508
Not Stated,225
Other,21


#### **QUESTION #14: What are the different types of collisions? Rank them according to the number of incidents.**

In [70]:
%%sql

SELECT 
    type_of_collision, 
    n_incidents,
    DENSE_RANK() OVER (ORDER BY n_incidents DESC) AS rank
FROM (
    SELECT 
        type_of_collision, 
        COUNT(*) AS n_incidents
    FROM 
        view_traffic_crash_all
    GROUP BY 
        type_of_collision);

 * sqlite:///traffic_crash.db
Done.


type_of_collision,n_incidents,rank
Broadside,17806,1
Vehicle/Pedestrian,12315,2
Rear End,9467,3
Sideswipe,7582,4
Head-On,3448,5
Other,3043,6
Hit Object,2221,7
Not Stated,1350,8
Overturned,1052,9


#### **QUESTION #15: Identify the type of victim with the highest number of fatalities.**
 * The number of deaths from the crash all data doesn't match the fatality data. The fatality data only records deaths from 2014 onwards.

In [101]:
%%sql

SELECT 
    count(*) AS fatality_total_deaths 
FROM traffic_crash_fatality;

 * sqlite:///traffic_crash.db
Done.


fatality_total_deaths
287


In [99]:
%%sql

SELECT 
    deceased, 
    COUNT(*) AS number_killed
FROM 
    traffic_crash_fatality
WHERE
    CAST(SUBSTR(collision_date, 1, 4) AS INTEGER) < 2024
GROUP BY 
    deceased
ORDER BY
    number_killed DESC;

 * sqlite:///traffic_crash.db
Done.


deceased,number_killed
Pedestrian,164
Motorcyclist,41
Driver,34
Bicyclist,22
Passenger,12
Standup Powered Device Rider,9
Exterior Passenger,3
Moped,2


In [97]:
%%sql

SELECT sum(number_killed) from view_traffic_crash_all limit 10;

 * sqlite:///traffic_crash.db
Done.


sum(number_killed)
559.0


In [106]:
%%sql

SELECT 
    MIN(collision_year) 
FROM 
    traffic_crash_fatality ;

 * sqlite:///traffic_crash.db
Done.


MIN(collision_year)
2014


#### **QUESTION #16: What is the percentage of incidents where there are no injured people?**

In [38]:
%%sql

WITH total_incidents AS (
    SELECT COUNT(*) AS total
    FROM view_traffic_crash_all
),
injured_zero_count AS (
    SELECT COUNT(*) AS zero_injured
    FROM view_traffic_crash_all
    WHERE number_injured = 0
)
SELECT
    ROUND((zero_injured * 1.0 / total)*100, 2) AS perc_no_injuries
FROM
    total_incidents, injured_zero_count;





 * sqlite:///traffic_crash.db
Done.


perc_no_injuries
0.96


# 4. File Creation
 * Create a csv with relevant columns for future data visualization. 

In [91]:
%%sql

DROP VIEW IF EXISTS view_at_fault_demographics;

CREATE VIEW view_at_fault_demographics AS
SELECT
    DISTINCT case_id_pkey,
    collision_date,
    party_type,
    at_fault, 
    party_sex,
    party_age
FROM 
    traffic_crash_victims
WHERE
    CAST(SUBSTR(collision_date, 7, 4) AS INTEGER) < 2024;
    




 * sqlite:///traffic_crash.db
Done.
Done.


[]

In [93]:
#output relevant data in csv files for data visualization

conn = sqlite3.connect('traffic_crash.db')
query1 = "SELECT * FROM view_at_fault_demographics"
at_fault_df = pd.read_sql_query(query1, conn)
at_fault_df.to_csv('Data/At_Fault_Demographics.csv', index=False)

query2 = "SELECT * FROM view_traffic_crash_all"
traffic_crash_df = pd.read_sql_query(query2, conn)
traffic_crash_df['primary_rd'] = traffic_crash_df['primary_rd'].str.title()
traffic_crash_df['secondary_rd'] = traffic_crash_df['secondary_rd'].str.title()
traffic_crash_df.to_csv('Data/Traffic_Crashes_All_Cleaned.csv', index=False)

conn.close()

In [45]:
'''%%sql

DROP VIEW IF EXISTS view_data_vis;

CREATE VIEW view_data_vis AS
SELECT
    t.unique_id,
    t.case_id_pkey,
    t.incident_year,
    t.collision_datetime,
    t.type_of_collision,
    t.number_killed,
    t.number_injured,
    t.dph_col_grp_description,
    t.primary_rd,
    t.secondary_rd,
    t.road_surface,
    t.lighting,
    t.tb_latitude,
    tb_longitude,
    v.victim_id,
    v.victim_sex,
    v.victim_age
FROM 
    view_traffic_crash_all AS t
LEFT JOIN
    (SELECT case_id_pkey, victim_id, victim_sex, victim_age 
     FROM traffic_crash_victims) AS v
ON t.case_id_pkey = v.case_id_pkey;'''


'%%sql\n\nDROP VIEW IF EXISTS view_data_vis;\n\nCREATE VIEW view_data_vis AS\nSELECT\n    t.unique_id,\n    t.case_id_pkey,\n    t.incident_year,\n    t.collision_datetime,\n    t.type_of_collision,\n    t.number_killed,\n    t.number_injured,\n    t.dph_col_grp_description,\n    t.primary_rd,\n    t.secondary_rd,\n    t.road_surface,\n    t.lighting,\n    t.tb_latitude,\n    tb_longitude,\n    v.victim_id,\n    v.victim_sex,\n    v.victim_age\nFROM \n    view_traffic_crash_all AS t\nLEFT JOIN\n    (SELECT case_id_pkey, victim_id, victim_sex, victim_age \n     FROM traffic_crash_victims) AS v\nON t.case_id_pkey = v.case_id_pkey;'